In [7]:
! pip install --quiet --upgrade langchain langchain-community langchain-chroma langchain-openai tenacity==8.5.0 python-dotenv gradio

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

langchain_api_key = os.getenv('LANGCHAIN_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')

os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
os.environ['OPENAI_API_KEY'] = openai_api_key

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'


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


In [11]:
from langchain_community.document_loaders import JSONLoader
import json
from pathlib import Path

file_path='./idea-2024-new.json'
data = json.loads(Path(file_path).read_text(encoding='utf-8'))
loader = JSONLoader(
         file_path=file_path,
         jq_schema=".[].summary",
         text_content=True)

In [4]:
docs = loader.load()

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

874

In [12]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [13]:
retriever = vectorstore.as_retriever()

In [14]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

import gradio as gr
import time

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

green_theme = gr.themes.Base(
    primary_hue=gr.themes.Color(
        c50="#00A168",     
        c100="#57B485",   
        c200="#D7ECE0",   
        c300="#FFFFFF",   
        c400="#EAE9E9",   
        c500="#000000",   
        c600="#3A905E",   
        c700="#2A774A",   
        c800="#1A5E36",   
        c900="#0A4512",   
        c950="#052A08"    
    ),
    font=[gr.themes.GoogleFont('Space Grotesk'), 'ui-sans-serif', 'system-ui', 'sans-serif']
).set(
    body_background_fill='#00A168',               
    body_text_color='#000000',                    
    background_fill_primary='#FFFFFF',            
    background_fill_secondary='#FFFFFF',          
    border_color_accent='#57B485',                
    border_color_accent_subdued='#EAE9E9',        
    color_accent='#57B485',                       
    color_accent_soft='#D7ECE0',                  
    checkbox_background_color='#FFFFFF',          
    button_primary_background_fill='#57B485',     
    button_primary_background_fill_hover='#3A905E', 
    button_secondary_background_fill='#D7ECE0',   
    button_secondary_text_color='#000000'         
)



# integrate gradio with RAG logic
def message_and_history(message, history):
    history = history or [{"role": "assistant", "content": "<b>LA2050 Navigator:</b><br> Welcome to the LA2050 ideas hub! How can I help you today?"}]    
    history.append({"role": "user", "content": message.get("text", "")})

    time.sleep(2)

    user_input = message.get("text", "")
    if not user_input:
        history.append({"role": "assistant", "content": "<b>LA2050 Navigator:</b><br> Please enter a valid message."})
        yield history, history
        return

    response = rag_chain.invoke({"input": user_input})
    answer = response["answer"]

    dynamic_message = {"role": "assistant", "content": "<b>LA2050 Navigator:</b><br> "}
    history.append(dynamic_message)

    for character in answer:
        dynamic_message["content"] += character
        time.sleep(0.05)
        yield history, history

    history[-1]["content"] = f"<b>LA2050 Navigator:</b><br> {answer}"
    yield history, history


# set to light mode
js_func = """
function refresh() {
    const url = new URL(window.location);

    if (url.searchParams.get('__theme') !== 'light') {
        url.searchParams.set('__theme', 'light');
        window.location.href = url.href;
    }
}
"""
css = """""
    .chat-header {
    text-color: #FFFFFF;
    text-align: center;
    }
    .gradio-container .prose .chat-header h1 {
    color: #FFFFFF;
    text-align: center; 
    }
   
}
"""

# setup gradio interface
with gr.Blocks(theme=green_theme, js=js_func, css=css) as block:

    gr.HTML('<div class="chat-header"><h1>LA2050 Navigator</h1></div>')
    
    chatbot = gr.Chatbot( value= 
                         [{"role": "assistant", "content": "<b>LA2050 Navigator:</b><br> Welcome to the LA2050 ideas hub! How can I help you today?"}],
                            type="messages",
                            bubble_full_width=False)
    
    state = gr.State([])

    message = gr.MultimodalTextbox(
        interactive=True,
        file_count="multiple",
        placeholder="Type a message", 
        label="", 
        elem_classes="custom-textbox", 
        scale=3,
        show_label=False)
    
    message.submit( 
        message_and_history, 
        inputs=[message, state],
        outputs=[chatbot, state]
    ).then(
        lambda: "", inputs=[], outputs=message  
    )

block.launch(debug=True, share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://1185ba6fddecb0e6b6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://1185ba6fddecb0e6b6.gradio.live
